In [1]:
import pandas as pd
import numpy as np

In [78]:
df = pd.read_csv('Churn_Modelling.csv')

In [87]:
df.head(20)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [79]:
DROP_COLUMNS = ['RowNumber', 'CustomerId', 'Surname']
df.drop(DROP_COLUMNS, inplace=True, axis=1)

In [80]:
# Mapping gender

df['Gender'] = df['Gender'].map({'Male': 1, 'Female':0})

In [88]:
#Dividing my population into controle and monitoramento

df_controle = df[(df['Geography'] == 'France') | (df['Geography'] == 'Spain')]
df_mon = df[df['Geography'] == 'Germany']

In [91]:
df_controle.drop('Geography', inplace=True, axis=1)
df_mon.drop('Geography', inplace=True, axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
#nans = np.random.randint(10,100,10)
#df.HasCrCard.iloc[nans] = np.nan

In [94]:
def scale_range (input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

buckets = 10
raw_breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

# Creating the breakpoints to every single column
breakpoints = pd.DataFrame()
for col in df_controle:
    breakpoints[col] = scale_range(raw_breakpoints, np.min(df_controle[col]), np.max(df_controle[col]))

In [95]:
breakpoints

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,350.0,0.0,18.0,0.0,0.000,1.0,0.0,0.0,90.070,0.0
1,400.0,0.1,25.4,1.0,25089.809,1.3,0.1,0.1,20080.311,0.1
2,450.0,0.2,32.8,2.0,50179.618,1.6,0.2,0.2,40070.552,0.2
3,500.0,0.3,40.2,3.0,75269.427,1.9,0.3,0.3,60060.793,0.3
4,550.0,0.4,47.6,4.0,100359.236,2.2,0.4,0.4,80051.034,0.4
5,600.0,0.5,55.0,5.0,125449.045,2.5,0.5,0.5,100041.275,0.5
6,650.0,0.6,62.4,6.0,150538.854,2.8,0.6,0.6,120031.516,0.6
7,700.0,0.7,69.8,7.0,175628.663,3.1,0.7,0.7,140021.757,0.7
8,750.0,0.8,77.2,8.0,200718.472,3.4,0.8,0.8,160011.998,0.8
9,800.0,0.9,84.6,9.0,225808.281,3.7,0.9,0.9,180002.239,0.9


In [97]:
# Distributing variables into buckets in the histogram
controle_hist = pd.DataFrame()
mon_hist = pd.DataFrame()

for col in df_controle:
    controle_hist[col] = np.histogram(df_controle[col], breakpoints[col])[0]
    mon_hist[col] = np.histogram(df_mon[col], breakpoints[col])[0]

In [98]:
controle_hist

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,13,3350,468,308,3623,3735,2227,3588,755,6268
1,126,0,1682,771,50,0,0,0,719,0
2,321,0,2794,1521,251,0,0,0,738,0
3,727,0,1344,0,748,3550,0,0,774,0
4,1087,0,557,757,1156,0,0,0,773,0
5,1386,0,371,753,1023,0,0,0,769,0
6,1492,0,163,740,477,170,0,0,754,0
7,1155,0,92,803,136,0,0,0,756,0
8,705,0,16,761,25,0,0,0,737,0
9,479,4141,4,1077,2,36,5264,3903,716,1223


In [99]:
mon_hist

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,6,1193,143,105,0,1349,718,1261,236,1695
1,40,0,497,264,19,0,0,0,249,0
2,126,0,835,536,109,0,0,0,269,0
3,231,0,527,0,425,1040,0,0,248,0
4,357,0,271,232,925,0,0,0,229,0
5,480,0,152,259,724,0,0,0,258,0
6,460,0,45,227,252,96,0,0,254,0
7,370,0,35,225,50,0,0,0,224,0
8,263,0,4,264,5,0,0,0,272,0
9,176,1316,0,397,0,24,1791,1248,269,814


In [100]:
# Getting percentages values
controle_hist = controle_hist / len(df_controle)
mon_hist = mon_hist / len(df_mon)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [104]:
for col in controle_hist:
    controle_hist[col][controle_hist[col] == 0] = 0.001
    mon_hist[col][mon_hist[col] == 0] = 0.001

In [105]:
# Calculating the PSI for every single column
df_psi = pd.DataFrame()
for col in controle_hist:
    df_psi['PSI_'+col] = (mon_hist[col] - controle_hist[col]) * np.log(mon_hist[col] / controle_hist[col])

In [110]:
df_psi

,PSI_CreditScore,PSI_Gender,PSI_Age,PSI_Tenure,PSI_Balance,PSI_NumOfProducts,PSI_HasCrCard,PSI_IsActiveMember,PSI_EstimatedSalary,PSI_Exited
0,0.000210,0.001735,0.000503,0.000013,2.983413,0.002947,0.000424,0.001137,4.645669e-04,0.034483
1,0.000047,0.000000,0.003315,0.000051,0.000113,0.000000,0.000000,0.000000,1.089446e-04,0.000000
2,0.001169,0.000000,0.004580,0.000538,0.002581,0.000000,0.000000,0.000000,7.355400e-04,0.000000
3,0.000262,0.000000,0.004827,0.000000,0.036751,0.007953,0.000000,0.000000,1.985688e-04,0.000000
4,0.000055,0.000000,0.012566,0.000763,0.186679,0.000000,0.000000,0.000000,1.462935e-03,0.000000
5,0.000210,0.000000,0.002228,0.000072,0.113711,0.000000,0.000000,0.000000,2.922818e-07,0.000000
6,0.001311,0.000000,0.000739,0.000730,0.016754,0.008132,0.000000,0.000000,3.349019e-06,0.000000
7,0.000299,0.000000,0.000213,0.003126,0.000165,0.000000,0.000000,0.000000,1.427107e-03,0.000000
8,0.001154,0.000000,0.000158,0.000128,0.000693,0.000000,0.000000,0.000000,9.727475e-04,0.000000
9,0.000575,0.001486,0.000292,0.001385,0.000968,0.003276,0.000175,0.001095,1.336003e-03,0.110678


In [112]:
pd.DataFrame(np.sum(df_psi), columns=['IEP'])

,IEP
PSI_CreditScore,0.005294
PSI_Gender,0.003220
PSI_Age,0.029421
PSI_Tenure,0.006806
PSI_Balance,3.341828
PSI_NumOfProducts,0.022309
PSI_HasCrCard,0.000599
PSI_IsActiveMember,0.002232
PSI_EstimatedSalary,0.006710
PSI_Exited,0.145161
